In [4]:
import datetime
import praw
import pandas as pd

from keys import client_id, secret

### Collecting posts

In [5]:
reddit = praw.Reddit(client_id=client_id, client_secret=secret, user_agent='postscrape by /u/kejimuna')

In [7]:
tesla = reddit.subreddit('teslamotors')

posts = []
for index, post in enumerate(tesla.top(limit=500)):
    posts.append([post.title, "https://reddit.com" + post.permalink, post.selftext, post.score, post.created_utc, index, post.num_comments])

# dataframe for posts
posts = pd.DataFrame(posts, columns=['Title', 'URL', 'Body', 'Upvotes', 'Time', 'Key', 'TotalComments'])

# changing from utc time to standard timestamp
posts.Time = posts.Time.apply(lambda x: pd.to_datetime(datetime.datetime.fromtimestamp(x)))

posts = posts.iloc[1:]

In [8]:
posts.head(5)

,Title,URL,Body,Upvotes,Time,Key,TotalComments
1,Elon Musk confirms secret Tesla ‘Carless Drive...,https://reddit.com/r/teslamotors/comments/7tvn...,,45609,2018-01-30 10:20:51,1,400
2,Tesla vs Bugatti,https://reddit.com/r/teslamotors/comments/7e2z...,,44272,2017-11-20 08:53:56,2,2352
3,Autopilot saves my model 3 from an accident!,https://reddit.com/r/teslamotors/comments/andk...,,39399,2019-02-06 01:20:41,3,1222
4,Update to the previous post,https://reddit.com/r/teslamotors/comments/7p6z...,,33412,2018-01-10 02:09:52,4,860
5,DAY ONE WITH MY TESLA!,https://reddit.com/r/teslamotors/comments/7yjr...,,32034,2018-02-19 16:27:47,5,998


Collecting the comments

In [44]:
def collect_replies(key, url):
    '''
    Args:
        key: the key of the post
        url (str): the url of the post

    Returns:
        pd.dataframe: comments
    '''

    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=None)
    comment_queue = submission.comments[:]

    table = {'Reply':[], 'Upvote':[], 'Time':[], 'Key':[]}

    while comment_queue:
        print(submission)
        comment = comment_queue.pop(0)
        table['Reply'].append(comment.body)
        table['Time'].append(comment.created_utc)
        table['Upvote'].append(comment.score)
        table['Key'].append(key)
        comment_queue.extend(comment.replies)

    return pd.DataFrame.from_dict(table)

In [24]:
comments.Time = comments.Time.apply(lambda x: pd.to_datetime(datetime.datetime.fromtimestamp(x)))

TypeError: an integer is required (got type Timestamp)

In [14]:
comments.shape

(226898, 4)

In [17]:
comments.to_csv('./data/raw/Comments.csv', index=False)

In [11]:
posts.to_csv('./data/raw/Posts.csv', index=False)